<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain two major functions within IllinoisGRMHD that are used to ensure that the results obtained throughout the simulation are Physically sound.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

If using the version of `IllinoisGRMHD` with piecewise polytropic *or* tabulated (coming soon!) EOS support, then the following citation is also required:

* **(Required)** Etienne, Z. B., Werneck, L., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L., *IllinoisGRMHD github repository* (2019). Source Code URL: https://github.com/zachetienne/nrpytutorial/tree/master/IllinoisGRMHD/.

### Dependencies

The files generated in this tutorial notebook depend on the following file:

* `IllinoisGRMHD_EoS_lowlevel_functs.C` \[[**tutorial**](Tutorial-IllinoisGRMHD__EoS_lowlevel_functs.ipynb)\]

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#introduction): **Introduction**
1. [Step 2](#apply_tau_floor): **The `apply_tau_floor` function**
    1. [Step 2.a](#positive_definiteness_of_the_metric): *Positive-definiteness of the metric*
    1. [Step 2.b](#barbi_barb_i_barb2_and_barb): *Computing $\bar{B}^{i}$, $\bar{B}_{i}$, $\bar{B}^{2}$, and $\bar{B}$*
    1. [Step 2.c](#barbdots_hatbarbdots_and_sdots): *Computing $\bar{B}\cdot\tilde{S}$, $\hat{\bar{B}}\cdot\tilde{S}$, and $\tilde{S}^{2}$*
    1. [Step 2.d](#modifying_tau): *Modifying $\tilde{\tau}$*
        1. [Step 2.d.i](#wm_sm_and_wmin): $W_{m}$, $\tilde{S}_{m}$, and $W_{\min}$
        1. [Step 2.d.ii](#tau_min): $\tau_{\min}$
    1. [Step 2.e](#modifying_tilde_s_i): *Modifying $\tilde{S}_{i}$*
1. [Step 3](#enforce_pressure_floor_ceiling) **The `enforce_pressure_floor_ceiling` function**
1. [Step 4](#enforce_limits_on_primitives_and_recompute_conservs): **The `IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs` function**
1. [Step 5](#code_validation): **Code validation**
1. [Step 6](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0b: Load up cmdline_helper and create the directory
import cmdline_helper as cmd
outdir = os.path.join("..","src")
cmd.mkdir(outdir)

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we will discuss how we adjust our conservative variables given that our primitive variables are in the physical range.

For a given set of primitive variables $\left\{\rho_{b},P,v^i,B^i\right\}$ in the physical range (i.e. $\rho_{b}\geq0$, $P\geq0$ and $\epsilon\geq0$), the corresponding conservative variables $\left\{\rho_{\star},\tilde{\tau},\tilde{S}_{i},\tilde{B}^{i}\right\}$ must satisfy certain inequalities (see appendix A of [Etienne *et al.* (2012)](https://arxiv.org/pdf/1112.0568.pdf) for the full discussion). Here we provide a practical recipe to impose these inequalities approximately to reduce inversion failures, which occur mainly in regions with very low density in the artificial “atmosphere” or inside the BH horizon where high accuracy is difficult to maintain but not crucial.

<a id='apply_tau_floor'></a>

# Step 2: The `apply_tau_floor` function \[Back to [top](#toc)\]
$$\label{apply_tau_floor}$$


Here we will start the `apply_tau_floor()` and declare a couple of function prototypes.

In [2]:
%%writefile $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

void eigenvalues_3by3_real_sym_matrix(CCTK_REAL & lam1, CCTK_REAL & lam2, CCTK_REAL & lam3,
                                      CCTK_REAL M11, CCTK_REAL M12, CCTK_REAL M13, CCTK_REAL M22, CCTK_REAL M23, CCTK_REAL M33);

static inline void enforce_pressure_floor_ceiling(output_stats &stats,CCTK_REAL kpoly,CCTK_REAL P_cold,CCTK_REAL Psi6,const CCTK_REAL Psi6threshold,CCTK_REAL rho_b,const CCTK_REAL rhobatm,  CCTK_REAL &P);

static inline int apply_tau_floor(const CCTK_REAL tau_atm,const CCTK_REAL rho_b_atm,const CCTK_REAL Psi6threshold,CCTK_REAL *PRIMS,CCTK_REAL *ADM_3METRIC,output_stats &stats,eos_struct &eos,  CCTK_REAL *CONSERVS) {

Overwriting ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='positive_definiteness_of_the_metric'></a>

## Step 2.a: Positive-definiteness of the metric \[Back to [top](#toc)\]
$$\label{positive_definiteness_of_the_metric}$$

We start by verifying if the metrix $\gamma_{ij}$ is positive definite. Notice that although we expect this to always be true, the metric may lose its positive-definiteness due to numerical error during the evolution, especially in the region deep inside the BH, near the “puncture”.

To verify whether or not the [metric is positive definite, we analyse its eigenvectors](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix#Eigenvalues). If the metrix is *not* positive definite, we reset $\gamma_{ij}\to\psi^{4}\tilde{\gamma}_{ij}$, where $\tilde{\gamma}_{ij}$ corresponds to the 3D flat metric tensor.

In [3]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


  //First apply the rho_star floor:

  //rho_star = alpha u0 Psi6 rho_b, alpha u0 > 1, so if rho_star < Psi6 rho_b_atm, then we are GUARANTEED that we can reset to atmosphere.
  //if(CONSERVS[RHOSTAR] < 1e4*ADM_3METRIC[SQRTGAMMA]*rho_b_atm) {
  //if(CONSERVS[RHOSTAR] < 2*ADM_3METRIC[SQRTGAMMA]*rho_b_atm) {

    CCTK_REAL CONF_METRIC[6];
    CCTK_REAL Psi2               = cbrt(ADM_3METRIC[SQRTGAMMA]);
    CCTK_REAL Psi4               = Psi2*Psi2;
    CCTK_REAL Psim4              = 1.0 / Psi4;
    CONF_METRIC[CM_GAMMATILDEXX] = ADM_3METRIC[GAMMAUPXX]*Psim4;
    CONF_METRIC[CM_GAMMATILDEXY] = ADM_3METRIC[GAMMAUPXY]*Psim4;
    CONF_METRIC[CM_GAMMATILDEXZ] = ADM_3METRIC[GAMMAUPXZ]*Psim4;
    CONF_METRIC[CM_GAMMATILDEYY] = ADM_3METRIC[GAMMAUPYY]*Psim4;
    CONF_METRIC[CM_GAMMATILDEYZ] = ADM_3METRIC[GAMMAUPYZ]*Psim4;
    CONF_METRIC[CM_GAMMATILDEZZ] = ADM_3METRIC[GAMMAUPZZ]*Psim4;
      
  CCTK_REAL lam1,lam2,lam3;
  eigenvalues_3by3_real_sym_matrix(lam1, lam2, lam3,CONF_METRIC[CM_GAMMATILDEXX], CONF_METRIC[CM_GAMMATILDEXY], CONF_METRIC[CM_GAMMATILDEXZ], CONF_METRIC[CM_GAMMATILDEYY], CONF_METRIC[CM_GAMMATILDEYZ], CONF_METRIC[CM_GAMMATILDEZZ]);
  if (lam1 < 0.0 || lam2 < 0.0 || lam3 < 0.0) {
    // Metric is not positive-defitive, reset the metric to be conformally-flat.
    CCTK_REAL Psi4               = cbrt(ADM_3METRIC[SQRTGAMMA]*ADM_3METRIC[SQRTGAMMA]);
    CONF_METRIC[CM_GAMMATILDEXX] = 1.0;
    CONF_METRIC[CM_GAMMATILDEXY] = 0.0;
    CONF_METRIC[CM_GAMMATILDEXZ] = 0.0;
    CONF_METRIC[CM_GAMMATILDEYY] = 1.0;
    CONF_METRIC[CM_GAMMATILDEYZ] = 0.0;
    CONF_METRIC[CM_GAMMATILDEZZ] = 1.0;
    ADM_3METRIC[GAMMAUPXX]       = Psi4;
    ADM_3METRIC[GAMMAUPXY]       = 0.0;
    ADM_3METRIC[GAMMAUPXZ]       = 0.0;
    ADM_3METRIC[GAMMAUPYY]       = Psi4;
    ADM_3METRIC[GAMMAUPYZ]       = 0.0;
    ADM_3METRIC[GAMMAUPZZ]       = Psi4;
  }

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='barbi_barb_i_barb2_and_barb'></a>

## Step 2.b: Computing $\bar{B}^{i}$, $\bar{B}_{i}$, $\bar{B}^{2}$, and $\bar{B}$ \[Back to [top](#toc)\]
$$\label{barbi_barb_i_barb2_and_barb}$$

We then set

$$
\boxed{\bar{B}^{i} = \frac{B^{i}}{\sqrt{4\pi}}}\ ,
$$

and

$$
\bar{B}_{i} = \gamma_{ij}\bar{B}^{j} \implies 
\boxed{
\left\{
\begin{matrix}
\bar{B}_{x} = \gamma_{xx}\bar{B}^{x} + \gamma_{xy}\bar{B}^{y} + \gamma_{xz}\bar{B}^{z}\\
\bar{B}_{y} = \gamma_{yx}\bar{B}^{x} + \gamma_{yy}\bar{B}^{y} + \gamma_{yz}\bar{B}^{z}\\
\bar{B}_{z} = \gamma_{zx}\bar{B}^{x} + \gamma_{zy}\bar{B}^{y} + \gamma_{zz}\bar{B}^{z}
\end{matrix}
\right.
}\ ,
$$

then

$$
\bar{B}^{2} \equiv B_{i}B^{i} \implies \boxed{\bar{B}^{2} = B_{x}B^{x} + B_{y}B^{y} + B_{z}B^{z}}\ ,
$$

and finally

$$
\boxed{\bar{B} \equiv \sqrt{\bar{B}^{2}}}\ .
$$

In [4]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


  //Next, prepare for the tau and stilde fixes:

  CCTK_REAL Bxbar = PRIMS[BX_CENTER]*ONE_OVER_SQRT_4PI,Bybar = PRIMS[BY_CENTER]*ONE_OVER_SQRT_4PI,Bzbar = PRIMS[BZ_CENTER]*ONE_OVER_SQRT_4PI;

  CCTK_REAL Bbar_x = ADM_3METRIC[GAMMAXX]*Bxbar + ADM_3METRIC[GAMMAXY]*Bybar + ADM_3METRIC[GAMMAXZ]*Bzbar;
  CCTK_REAL Bbar_y = ADM_3METRIC[GAMMAXY]*Bxbar + ADM_3METRIC[GAMMAYY]*Bybar + ADM_3METRIC[GAMMAYZ]*Bzbar;
  CCTK_REAL Bbar_z = ADM_3METRIC[GAMMAXZ]*Bxbar + ADM_3METRIC[GAMMAYZ]*Bybar + ADM_3METRIC[GAMMAZZ]*Bzbar;
  CCTK_REAL Bbar2 = Bxbar*Bbar_x + Bybar*Bbar_y + Bzbar*Bbar_z;
  CCTK_REAL Bbar = sqrt(Bbar2);

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


The next part of the code is written to prevent [floating-point underflow](https://en.wikipedia.org/wiki/Arithmetic_underflow). We compute $\bar{B}$ in a different way. We start by evaluating

$$
\bar{B}_{\rm check} = \left|\bar{B}^{x}\right| + \left|\bar{B}^{y}\right| + \left|\bar{B}^{z}\right|\ ,
$$

and verifying whether that is a very small, positive number. Then, we determine the largest component of $\bar{B}_{\rm check}$: 

$$
\bar{B}_{\max} = \max\left(\left|\bar{B}^{x}\right|,\left|\bar{B}^{y}\right|,\left|\bar{B}^{z}\right|\right)\ .
$$

Then, we rescale $\bar{B}_{i}$ and $\bar{B}^{i}$ using

$$
\left(\bar{B}^{i}\right)_{\rm tmp} \equiv \frac{\bar{B}^{i}}{\bar{B}_{\max}}\ ,\quad
\left(\bar{B}_{i}\right)_{\rm tmp} \equiv \frac{\bar{B}_{i}}{\bar{B}_{\max}}\ ,
$$

and finally recompute $\bar{B}$

$$
\bar{B} = \left[\left(\bar{B}_{i}\right)_{\rm tmp}\left(\bar{B}^{i}\right)_{\rm tmp}\right]\bar{B}_{\max}\ .
$$

In [5]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

  CCTK_REAL check_B_small = fabs(Bxbar)+fabs(Bybar)+fabs(Bzbar);
  if (check_B_small>0 && check_B_small<1.e-150) {
    // need to compute Bbar specially to prevent floating-point underflow
    CCTK_REAL Bmax = fabs(Bxbar);
    if (Bmax < fabs(Bybar)) Bmax=fabs(Bybar);
    if (Bmax < fabs(Bzbar)) Bmax=fabs(Bzbar);
    CCTK_REAL Bxtmp=Bxbar/Bmax, Bytemp=Bybar/Bmax, Bztemp=Bzbar/Bmax;
    CCTK_REAL B_xtemp=Bbar_x/Bmax, B_ytemp=Bbar_y/Bmax, B_ztemp=Bbar_z/Bmax;
    Bbar = sqrt(Bxtmp*B_xtemp + Bytemp*B_ytemp + Bztemp*B_ztemp)*Bmax;
  }

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='barbdots_hatbarbdots_and_sdots'></a>

## Step 2.c: Computing $\bar{B}\cdot\tilde{S}$, $\hat{\bar{B}}\cdot\tilde{S}$, and $\tilde{S}^{2}$ \[Back to [top](#toc)\]
$$\label{barbdots_hatbarbdots_and_sdots}$$

Then we compute

$$
\bar{B} \cdot \tilde{S} = \bar{B}_{i}\tilde{S}^{i} = \bar{B}_{x}\tilde{S}^{x} + \bar{B}_{y}\tilde{S}^{y} + \bar{B}_{z}\tilde{S}^{z}\ .
$$

and

$$
\hat{\bar{B}}\cdot\tilde{S} \equiv \frac{\bar{B} \cdot \tilde{S}}{\bar{B}}\ .
$$

However, if $\bar{B} \ll 1$, we set $\hat{\bar{B}}\cdot\tilde{S}=0$.

In [6]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

  CCTK_REAL BbardotS = Bxbar*CONSERVS[STILDEX] + Bybar*CONSERVS[STILDEY] + Bzbar*CONSERVS[STILDEZ];
  CCTK_REAL hatBbardotS = BbardotS/Bbar;
  if (Bbar<1.e-300) hatBbardotS = 0.0;

  // Limit hatBbardotS
  //CCTK_REAL max_gammav = 100.0;
  //CCTK_REAL rhob_max = CONSERVS[RHOSTAR]/ADM_3METRIC[SQRTGAMMA];
  //CCTK_REAL hmax = 1.0 + 2.0*rhob_max;
  //CCTK_REAL abs_hatBbardotS_max = sqrt(SQR(max_gammav)-1.0)*CONSERVS[RHOSTAR]*hmax;
  //if (fabs(hatBbardotS) > abs_hatBbardotS_max) {
  //   CCTK_REAL fac_reduce = abs_hatBbardotS_max/fabs(hatBbardotS);
  //   CCTK_REAL hatBbardotS_max = hatBbardotS*fac_reduce;
  //   CCTK_REAL Bbar_inv = 1.0/Bbar;
  //   CCTK_REAL hat_Bbar_x = Bbar_x*Bbar_inv;
  //   CCTK_REAL hat_Bbar_y = Bbar_y*Bbar_inv;
  //   CCTK_REAL hat_Bbar_z = Bbar_z*Bbar_inv;
  //   CCTK_REAL sub_fact = hatBbardotS_max - hatBbardotS;
  //   CONSERVS[STILDEX] += sub_fact*hat_Bbar_x;
  //   CONSERVS[STILDEY] += sub_fact*hat_Bbar_y;
  //   CONSERVS[STILDEZ] += sub_fact*hat_Bbar_z;
  //   hatBbardotS = hatBbardotS_max;
  //   BbardotS *= fac_reduce;
  //   CONSERVS[STILDEX] = CONSERVS[STILDEX]; CONSERVS[STILDEY] = CONSERVS[STILDEY]; CONSERVS[STILDEZ] = CONSERVS[STILDEZ];
  //}

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


Next we compute

$$
\tilde{S}^{2} \equiv \tilde{S} \cdot \tilde{S} = \gamma^{ij}\tilde{S}_{i}\tilde{S}_{j}\ ,
$$

i.e.

$$
\boxed{
\begin{align}
\tilde{S}^{2} &= \gamma^{xx}\left(\tilde{S}_{x}\right)^{2}
                           + \gamma^{yy}\left(\tilde{S}_{y}\right)^{2}
                           + \gamma^{zz}\left(\tilde{S}_{z}\right)^{2}\\
                          &+2\left(
                             \gamma^{xy}\tilde{S}_{x}\tilde{S}_{y}
                            +\gamma^{xz}\tilde{S}_{x}\tilde{S}_{z}
                            +\gamma^{yz}\tilde{S}_{y}\tilde{S}_{z}
                           \right)
\end{align}
}\ .
$$

In [7]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


  CCTK_REAL sdots= ADM_3METRIC[GAMMAUPXX]*SQR(CONSERVS[STILDEX])+ADM_3METRIC[GAMMAUPYY]*SQR(CONSERVS[STILDEY])+ADM_3METRIC[GAMMAUPZZ]*SQR(CONSERVS[STILDEZ])+2.0*
    (ADM_3METRIC[GAMMAUPXY]*CONSERVS[STILDEX]*CONSERVS[STILDEY]+ADM_3METRIC[GAMMAUPXZ]*CONSERVS[STILDEX]*CONSERVS[STILDEZ]+ADM_3METRIC[GAMMAUPYZ]*CONSERVS[STILDEY]*CONSERVS[STILDEZ]);

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='modifying_tau'></a>

## Step 2.d: Modifying $\tilde{\tau}$ \[Back to [top](#toc)\]
$$\label{modifying_tau}$$

<a id='wm_sm_and_wmin'></a>

### Step 2.d.i: $W_{m}$, $\tilde{S}_{m}$, and $W_{\min}$ \[Back to [top](#toc)\]
$$\label{wm_sm_and_wmin}$$

Then we compute other useful quantities, which are eqs. (A52), (A53), and (A54) in appendix A of [Etienne *et al.* (2012)](https://arxiv.org/pdf/1112.0568.pdf)

$$
\begin{align}
W_{m} &= \psi^{-6}\left[\left(\hat{\bar{B}}\cdot\tilde{S}\right)^{2}+\rho_{\star}^{2}\right]^{1/2}\ ,\\
\tilde{S}_{m}^{2} &= \frac{W_{m}^{2}\tilde{S}^{2} + \left(\bar{B}\cdot\tilde{S}\right)^{2}\left(\bar{B}^{2}+2W_{m}\right)}{\left(W_{m}+\bar{B}^{2}\right)^{2}}\ ,\\
W_{\min} &= \psi^{-6}\left(S_{m}^{2}+\rho_{\star}^{2}\right)^{1/2}\ ,\\
\end{align}
$$

respectively (notice the slightly different notation between the equations above and the one used in the paper).

In [8]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


  CCTK_REAL Wm = sqrt(SQR(hatBbardotS)+ SQR(CONSERVS[RHOSTAR]))/ADM_3METRIC[SQRTGAMMA];
  CCTK_REAL Sm2 = (SQR(Wm)*sdots + SQR(BbardotS)*(Bbar2+2.0*Wm))/SQR(Wm+Bbar2);
  CCTK_REAL Wmin = sqrt(Sm2 + SQR(CONSERVS[RHOSTAR]))/ADM_3METRIC[SQRTGAMMA];
  CCTK_REAL sdots_fluid_max = sdots;

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='tau_min'></a>

### Step 2.d.ii: $\tilde{\tau}_{\min}$ \[Back to [top](#toc)\]
$$\label{tau_min}$$

Next we evaluate

$$
\tilde{\tau}_{\min} = \tilde{\tau} - \frac{\psi^{6}}{2}\bar{B}^{2} - \frac{\bar{B}^{2}\tilde{S}^{2} - \left(\bar{B}\cdot\tilde{S}\right)^{2}}{2\psi^{6}\left(W_{\min}+\bar{B}^{2}\right)^{2}}
$$

In [9]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


  //tau fix, applicable when B==0 and B!=0:
  if(CONSERVS[TAUENERGY] < 0.5*ADM_3METRIC[SQRTGAMMA]*Bbar2) {
    CONSERVS[TAUENERGY] = tau_atm+0.5*ADM_3METRIC[SQRTGAMMA]*Bbar2;
    stats.failure_checker+=1000000;
  }

  CCTK_REAL tau_fluid_min = CONSERVS[TAUENERGY] - 0.5*ADM_3METRIC[SQRTGAMMA]*Bbar2 - (Bbar2*sdots - SQR(BbardotS))*0.5/(ADM_3METRIC[SQRTGAMMA]*SQR(Wmin+Bbar2));

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


Then we verify if $\tilde{\tau}_{\min} \geq \tilde{\tau}_{\rm atm}$. If $\tilde{\tau}_{\min} < \tilde{\tau}_{\rm atm}$, then reset $\tilde{\tau}$:

$$
\tilde{\tau} = \tilde{\tau}_{\min} + \frac{\psi^{6}}{2}\bar{B}^{2} + \frac{\bar{B}^{2}\tilde{S}^{2} - \left(\bar{B}\cdot\tilde{S}\right)^{2}}{2\psi^{6}\left(W_{\min}+\bar{B}^{2}\right)^{2}}\ .
$$

In [10]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

  
  //Apply Stilde fix when B==0.
  //if(PRIMS[BX_CENTER]==0 && PRIMS[BY_CENTER]==0 && PRIMS[BZ_CENTER]==0 && (ADM_3METRIC[SQRTGAMMA]>30.0 || CONSERVS[RHOSTAR]/ADM_3METRIC[SQRTGAMMA]<100*rho_b_atm)) {
  //if(check_B_small < 1.e-300) {
      
  /**********************************
   * Piecewise Polytropic EOS Patch *
   *         Computing Patm         *
   **********************************/
  /* This modification of the code trades the variable
   * "gamma_equals2" by the already defined function
   * pow().
   * 
   * Also, assuming that Patm < rho_ppoly_tab[0], we skip 
   * the declaration of new variables to store the 
   * values of K_ppoly_tab[0] and Gamma_ppoly_tab[0]. Thus:
   *  -----------------------------------------
   * | P_{atm} = K_{0} * rho_{atm}^{Gamma_{0}} |
   *  -----------------------------------------
   */
  int polytropic_index = find_polytropic_K_and_Gamma_index(eos,rho_b_atm);
  CCTK_REAL Patm = eos.K_ppoly_tab[polytropic_index]*pow(rho_b_atm,eos.Gamma_ppoly_tab[polytropic_index]);

  if(check_B_small*check_B_small < Patm*1e-32) {
    CCTK_REAL rhot=CONSERVS[TAUENERGY]*(CONSERVS[TAUENERGY]+2.0*CONSERVS[RHOSTAR]);
    CCTK_REAL safetyfactor = 0.999999;
    //if(ADM_3METRIC[SQRTGAMMA]>Psi6threshold) safetyfactor=0.99;

    if(sdots > safetyfactor*rhot) {
      CCTK_REAL rfactm1 = sqrt((safetyfactor*rhot)/sdots);
      CONSERVS[STILDEX]*=rfactm1;
      CONSERVS[STILDEY]*=rfactm1;
      CONSERVS[STILDEZ]*=rfactm1; 
      stats.failure_checker+=10000000;
    }
  } else if(ADM_3METRIC[SQRTGAMMA]>Psi6threshold) {
    //Apply new Stilde fix.
    if (tau_fluid_min < tau_atm*1.001) { 
      tau_fluid_min = tau_atm*1.001;
      CONSERVS[TAUENERGY] = tau_fluid_min + 0.5*ADM_3METRIC[SQRTGAMMA]*Bbar2 + (Bbar2*sdots - SQR(BbardotS))*0.5/(ADM_3METRIC[SQRTGAMMA]*SQR(Wmin+Bbar2));
    }

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='modifying_tilde_s_i'></a>

## Step 2.e: Modifying $\tilde{S}_{i}$ \[Back to [top](#toc)\]
$$\label{modifying_tilde_s_i}$$

Then we check if $\tilde{S}^{2} \leq \tilde{\tau}_{\min}\left(\tilde{\tau}_{\min}+2\rho_{\star}\right)$. If not, we reset $\tilde{S}_{i}$

$$
\tilde{S}_{i}\to \tilde{S}_{i}\sqrt{\frac{\tilde{\tau}_{\min}\left(\tilde{\tau}_{\min}+2\rho_{\star}\right)}{\tilde{S}^{2}}}
$$

In [11]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

    CCTK_REAL LHS = tau_fluid_min*(tau_fluid_min+2.0*CONSERVS[RHOSTAR]);
    CCTK_REAL RHS = sdots_fluid_max;

    CCTK_REAL safetyfactor = 0.999999;
    if(safetyfactor*LHS < RHS) {
      CCTK_REAL rfactm1 = sqrt((safetyfactor*LHS)/RHS);
      CONSERVS[STILDEX]*=rfactm1;
      CONSERVS[STILDEY]*=rfactm1;
      CONSERVS[STILDEZ]*=rfactm1;
      stats.failure_checker+=100000000;
    }
  }
  


  return 0;
}

/***********************************************************/
/***********************************************************/
/***********************************************************/
/***********************************************************/



Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='enforce_pressure_floor_ceiling'></a>

# Step 3: The `enforce_pressure_floor_ceiling` function \[Back to [top](#toc)\]
$$\label{enforce_pressure_floor_ceiling}$$

After the Newton-Raphson solver has successfully found a set of primitives, the primitives are checked for physicality, and if they are not in the physical range, they are minimally modified until they return to the physical range. First,if the velocity is found to be superluminal, the speed is reduced to `IllinoisGRMHD`’s default Lorentz factor limit, a procedure which we already explained above when we discussed the `impose_speed_limit_output_u0` function.

Next, `IllinoisGRMHD` does not include any cooling mechanism, which means that for evolutions adopting a $\Gamma$-law equation of state, the pressure should not physically drop below $P_{\rm cold}$. So a pressure floor of $0.9P_{\rm cold}$ is imposed. Increasing this floor to $P_{\rm cold}$ exactly results in large central density drifts in TOV star evolutions.

**NOTE**: Please keep in mind that the floor and ceiling values presented here were found ***empirically***.

In [12]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


static inline void enforce_pressure_floor_ceiling(output_stats &stats,CCTK_REAL kpoly,CCTK_REAL P_cold,CCTK_REAL Psi6,const CCTK_REAL Psi6threshold,CCTK_REAL rho_b,const CCTK_REAL rhobatm,  CCTK_REAL &P) {
  CCTK_REAL P_min=0.9*P_cold;
  if(P<P_min) {
    stats.failure_checker+=10;
    P=P_min;
  }
  //MAX(P,P_min);
  //if(P < P_min) P=1.0*P_cold;

  /* OLD: Discarded because lower limit is unphysical.
     if(P <= 0.5*kpoly*P_cold) {
     P=0.5*kpoly*P_cold;
     }
  */

Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


Simulations can crash in the other extreme, if $P/P_{\rm cold}$ becomes too large. This typically only happens in very low density regions or inside black holes.  So at densities $\rho_{b}<100\rho_{\rm atm}$ or deep inside black hole horizons, a ceiling on $P$ of $100P_{\rm cold}$ is enforced (see Appendix A of [Etienne *et al.* (2012)](https://arxiv.org/abs/1112.0568) for more details).

We also introduce a parameter, $\psi^{6}_{\rm threshold}$, which determines whether the region under consideration is deep inside the BH horizon or not. For regions deep inside the BH horizon, defined by $\sqrt{\gamma} = \psi^{6} > \psi^{6}_{\rm threshold}$, the primary goal is to keep the evolution stable and prevent inaccurate data from leaking out of the BH horizon. It was determined that in this situation, a better ceiling on $P$ is $10^{5}P_{\rm cold}$.

In [13]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


  //CCTK_REAL P_max = 10.0*P_cold;
  CCTK_REAL P_max = 100.0*P_cold;
  if(Psi6 > Psi6threshold) P_max = 1e5*P_cold; // <-- better than 10.

  if((rho_b < 100.0*rhobatm || Psi6 > Psi6threshold) && P>P_max) {
    P=P_max;
    stats.failure_checker+=100;
  }

  /*
    CCTK_REAL rho_horiz_cap = 1000.0*rhobatm;
    
    //New density damping mechanism inside the horizon
    if(Psi6 > Psi6threshold && rho_b>rho_horiz_cap) {
    CCTK_REAL six_phi=log(Psi6);
    CCTK_REAL six_phithreshold=log(Psi6threshold);
    CCTK_REAL Psi6max_approx=350000;
    rho_b = rho_horiz_cap+(rho_b-rho_horiz_cap)*exp(-200.0*SQR((six_phi-six_phithreshold)/log(Psi6max_approx)));
    }
  */
}



Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='enforce_limits_on_primitives_and_recompute_conservs'></a>

# Step 4: `The IllinoisGRMHD_ enforce_limits_on_primitives_and_recompute_conservs` function \[Back to [top](#toc)\]
$$\label{enforce_limits_on_primitives_and_recompute_conservs}$$

We start by imposing physical limits on the primitive variables $\left\{\rho_{b},P,v^{i}\right\}$, using:

1. $\rho_{b} \to \min\left(\rho_{b},\rho_{b,{\rm atm}}\right)$
1. `enforce_pressure_floor_ceiling()`: documented [above](#enforce_pressure_floor_ceiling)

We then compute $g_{\mu\nu}$, $g^{\mu\nu}$, $T_{\mu\nu}$, $T^{\mu\nu}$, and, finally, recompute the conservative variables.

In this step, we will NRPy+ to generate expressions for:

1. The physical ADM 4-metric, $g_{\mu\nu}$: done using the [BSSN.ADMBSSN_tofrom_4metric NRPy+ module](/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-ADMBSSN_tofrom_4metric.ipynb)\]
2. The physical ADM inverse 4-metric $g^{\mu\nu}$: done using the [BSSN.ADMBSSN_tofrom_4metric NRPy+ module](/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-ADMBSSN_tofrom_4metric.ipynb)\]
3. The covariant GRMHD energy momentum tensor, $T_{\mu\nu}^{\rm GRMHD}$, which is done is three steps:
    1. Compute $T_{\mu\nu}^{\rm GRHD}$: done using the [GRHD.equations NRPy+ module](/edit/NRPyIGM/GRHD/equations.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-GRHD_Equations-Cartesian.ipynb)\]
    2. Compute $T_{\mu\nu}^{\rm GRFFE}$: done using the [GRFFE.equations NRPy+ module](/edit/NRPyIGM/GRFFE/equations.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-GRFFE_Equations-Cartesian.ipynb)\]
    3. Compute $T_{\mu\nu}^{\rm GRMHD} = T_{\mu\nu}^{\rm GRHD} + T_{\mu\nu}^{\rm GRFFE}$: done using the [GRMHD.equations NRPy+ module](/edit/NRPyIGM/GRMHD/equations.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-GRMHD_Equations-Cartesian.ipynb)\]
4. The contravariant GRMHD energy momentum tensor, $T^{\mu\nu}_{\rm GRMHD}$, which is done is three steps:
    1. Compute $T^{\mu\nu}_{\rm GRHD}$: done using the [GRHD.equations NRPy+ module](/edit/NRPyIGM/GRHD/equations.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-GRHD_Equations-Cartesian.ipynb)\]
    2. Compute $T^{\mu\nu}_{\rm GRFFE}$: done using the [GRFFE.equations NRPy+ module](/edit/NRPyIGM/GRFFE/equations.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-GRFFE_Equations-Cartesian.ipynb)\]
    3. Compute $T^{\mu\nu}_{\rm GRMHD} = T^{\mu\nu}_{\rm GRHD} + T^{\mu\nu}_{\rm GRFFE}$: done using the [GRMHD.equations NRPy+ module](/edit/NRPyIGM/GRMHD/equations.py) \[[**tutorial**](/notebooks/NRPyIGM/Tutorial-GRMHD_Equations-Cartesian.ipynb)\]

In [14]:
from outputC import *
import indexedexp as ixp
import sympy as sp
import BSSN.ADMBSSN_tofrom_4metric as AB4m
import GRHD.equations as GRHD
import GRFFE.equations as GRFFE
import GRMHD.equations as GRMHD

NRPy_headers_dir_path = os.path.join(outdir,"NRPy_generated_headers")

ADMgammaDD = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        ADMgammaDD[i][j] = ADMgammaDD[j][i] = sp.symbols("ADM_3METRIC[GAMMA"+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)
ADMbetaU = ixp.zerorank1()
for i in range(3):
    ADMbetaU[i] = sp.symbols("ADM_3METRIC[BETA"+chr(ord('X')+i)+"]",real=True)
ADMalpha = sp.symbols("ADM_3METRIC[ALPHA]",real=True)
vU = ixp.zerorank1()
for i in range(3):
    vU[i] = sp.symbols("PRIMS[V"+chr(ord('X')+i)+"]",real=True)

GRHD.u4U_in_terms_of_vU__rescale_vU_by_applying_speed_limit(ADMalpha,ADMbetaU,ADMgammaDD, vU)
for i in range(3):
    vU[i] = GRHD.rescaledvU[i]
u4U = GRHD.u4U_ito_vU

# First compute smallb4U & smallbsquared from BtildeU, which are needed
#      for GRMHD stress-energy tensor T4UU and T4UD:
GRHD.compute_sqrtgammaDET(ADMgammaDD)
B_notildeU = ixp.zerorank1()
for i in range(3):
    B_notildeU[i] = sp.symbols("PRIMS[B"+chr(ord('X')+i)+"_CENTER]",real=True)

sqrt4pi = sp.symbols('sqrt4pi', real=True)
GRFFE.compute_smallb4U(     ADMgammaDD,ADMbetaU,ADMalpha, u4U,B_notildeU, sqrt4pi)
GRFFE.compute_smallbsquared(ADMgammaDD,ADMbetaU,ADMalpha, GRFFE.smallb4U)

rho_b,P,epsilon = sp.symbols("PRIMS[RHOB] PRIMS[PRESSURE] eps", real=True)

GRMHD.compute_GRMHD_T4UU(ADMgammaDD, ADMbetaU, ADMalpha, rho_b, P, epsilon, u4U, 
                         GRFFE.smallb4U, GRFFE.smallbsquared)

GRMHD.compute_GRMHD_T4UD(ADMgammaDD, ADMbetaU, ADMalpha, GRMHD.GRHDT4UU,GRMHD.GRFFET4UU)
# Compute g_{\mu\nu}
AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=ADMgammaDD,betaU=ADMbetaU,alpha=ADMalpha)
ADMgammaUU,dummy = ixp.symm_matrix_inverter3x3(ADMgammaDD)
AB4m.g4UU_ito_BSSN_or_ADM("ADM",betaU=ADMbetaU,alpha=ADMalpha,gammaUU=ADMgammaUU)

# Set up g4DD and T4UU in a way that uses the already computed
# gridfunctions instead of the complicated SymPy expressions
g4DD = ixp.zerorank2(DIM=4)
T4UU = ixp.zerorank2(DIM=4)
count = 0
for mu in range(4):
    for nu in range(mu,4):
        g4DD[mu][nu] = g4DD[nu][mu] = sp.Symbol("g4dn["+str(mu)+"]["+str(nu)+"]",real=True)
        T4UU[mu][nu] = T4UU[nu][mu] = sp.Symbol("TUPMUNU["+str(count)+"]",real=True)
        count += 1
        
# Compute T4DD using the simplified expressions
T4DD = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        for rho in range(4):
            for sigma in range(4):
                T4DD[mu][nu] += g4DD[mu][rho] * g4DD[nu][sigma] * T4UU[rho][sigma]

# Compute conservative variables in terms of primitive variables
GRHD.compute_rho_star( ADMalpha, GRHD.sqrtgammaDET, rho_b,u4U)
GRHD.compute_tau_tilde(ADMalpha, GRHD.sqrtgammaDET, GRMHD.T4UU,GRHD.rho_star)
GRHD.compute_S_tildeD( ADMalpha, GRHD.sqrtgammaDET, GRMHD.T4UD)

varlist = []
exprlist = []

for mu in range(4):
    for nu in range(4):
        varlist.append("g4dn["+str(mu)+"]["+str(nu)+"]")
        exprlist.append(AB4m.g4DD[mu][nu])

for mu in range(4):
    for nu in range(4):
        varlist.append("g4up["+str(mu)+"]["+str(nu)+"]")
        exprlist.append(AB4m.g4UU[mu][nu])

count = 0
for mu in range(4):
    for nu in range(mu,4):
        varlist.append("TUPMUNU["+str(count)+"]")
        exprlist.append(GRMHD.T4UU[mu][nu])
        count += 1
        
import time
start = time.time()
outputC(exprlist,varlist,filename=os.path.join(NRPy_headers_dir_path,"compute_g4dn_g4up_T4UU.h"),
        params="outCverbose=False")
print("Time elapsed: %4.2lf seconds"%(time.time()-start))
        
varlist = []
exprlist = []

count = 0
for mu in range(4):
    for nu in range(mu,4):
        varlist.append("TDNMUNU["+str(count)+"]")
        exprlist.append(T4DD[mu][nu])
        count += 1

varlist.append("CONSERVS[RHOSTAR]")
exprlist.append(GRHD.rho_star)
for i in range(3):
    varlist.append("CONSERVS[STILDE"+chr(ord('X')+i)+"]")
    exprlist.append(GRHD.S_tildeD[i])
varlist.append("CONSERVS[TAUENERGY]")
exprlist.append(GRHD.tau_tilde)

# for mu in range(4):
#     varlist.append("smallb4U["+str(mu)+"]")
#     exprlist.append(GRFFE.smallb4U[mu])

# varlist.append("smallbsquared")
# exprlist.append(GRFFE.smallbsquared)

start = time.time()
outputC(exprlist,varlist,filename=os.path.join(NRPy_headers_dir_path,"compute_T4DD_CONSERVS.h"),
        params="outCverbose=False")
print("Time elapsed: %4.2lf seconds"%(time.time()-start))

Wrote to file "../src/NRPy_generated_headers/compute_g4dn_g4up_T4UU.h"
Time elapsed: 4.62 seconds
Wrote to file "../src/NRPy_generated_headers/compute_T4DD_CONSERVS.h"
Time elapsed: 11.80 seconds


In [15]:
%%writefile -a $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

void IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs(const int already_computed_physical_metric_and_inverse,CCTK_REAL *PRIMS,
                                                                       struct output_stats &stats,eos_struct &eos,
                                                                       CCTK_REAL *ADM_3METRIC,CCTK_REAL g4dn[4][4],CCTK_REAL g4up[4][4],
                                                                       CCTK_REAL *TUPMUNU,CCTK_REAL *TDNMUNU,CCTK_REAL *CONSERVS) {
#ifndef ENABLE_STANDALONE_IGM_C2P_SOLVER
  DECLARE_CCTK_PARAMETERS;
#endif
  
  // Useful debugging tool, can be used to track fixes:
  //CCTK_REAL rho_b_orig=PRIMS[RHOB],P_orig=PRIMS[PRESSURE],vx_orig=PRIMS[VX],vy_orig=PRIMS[VY],vz_orig=PRIMS[VZ];

  /***********************************************************/
  // Enforce limits on pressure, density, and v^i
  /***********************************************************/
  // Density floor:
  //  printf("HEY222a %e %e %e\n",PRIMS[RHOB],rho_b_atm,PRIMS[PRESSURE]);
  PRIMS[RHOB] = MAX(PRIMS[RHOB],rho_b_atm);
  // Density ceiling:
  PRIMS[RHOB] = MIN(PRIMS[RHOB],rho_b_max);
  //   Next set h, the enthalpy:
  CCTK_REAL h_enthalpy,  P_cold,eps_cold,dPcold_drho,eps_th,Gamma_cold; /* <- Note that in setting h, we need to define several 
                                                                         *    other variables. Though some will be unused later
                                                                         *    in this function, they may be useful in other
                                                                         *    functions */
  compute_P_cold__eps_cold__dPcold_drho__eps_th__h__Gamma_cold(PRIMS,eos,Gamma_th,P_cold,eps_cold,dPcold_drho,eps_th,h_enthalpy,Gamma_cold);

  // Pressure floor & ceiling:
  int polytropic_index = find_polytropic_K_and_Gamma_index(eos,PRIMS[RHOB]);
  enforce_pressure_floor_ceiling(stats,eos.K_ppoly_tab[polytropic_index],P_cold,ADM_3METRIC[SQRTGAMMA],Psi6threshold,PRIMS[RHOB],rho_b_atm,  PRIMS[PRESSURE]);
  //  printf("HEY222b %e %e %e\n",PRIMS[RHOB],rho_b_atm,PRIMS[PRESSURE]);

  // Possibly adjusted pressure, so recompute eps & h:
  CCTK_REAL eps = eps_cold + (PRIMS[PRESSURE]-P_cold)/(Gamma_th-1.0)/PRIMS[RHOB];
  const CCTK_REAL sqrt4pi = 1.0 * sqrt(4.0*M_PI);
  CCTK_REAL smallb4U[4];
  CCTK_REAL smallbsquared;
  //FIXME: Use already_computed_physical_metric_and_inverse to determine whether g4dn & g4up really need to be calculated.
#include "NRPy_generated_headers/compute_g4dn_g4up_T4UU.h"
#include "NRPy_generated_headers/compute_T4DD_CONSERVS.h"

  //printf("HEY?? %e %e %e %e %e\n",smallb4U[0],smallb4U[1],smallb4U[2],smallb4U[3],smallbsquared);
}


Appending to ../src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C


<a id='code_validation'></a>

# Step 5: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [16]:
# # Verify if the code generated by this tutorial module
# # matches the original IllinoisGRMHD source code

# # First download the original IllinoisGRMHD source code
# import urllib
# from os import path

# original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C"
# original_IGM_file_name = "apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs-original.C"
# original_IGM_file_path = os.path.join(outdir,original_IGM_file_name)

# # Then download the original IllinoisGRMHD source code
# # We try it here in a couple of ways in an attempt to keep
# # the code more portable
# try:
#     original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
#     # Write down the file the original IllinoisGRMHD source code
#     with open(original_IGM_file_path,"w") as file:
#         file.write(original_IGM_file_code)
# except:
#     try:
#         original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
#         # Write down the file the original IllinoisGRMHD source code
#         with open(original_IGM_file_path,"w") as file:
#             file.write(original_IGM_file_code)
#     except:
#         # If all else fails, hope wget does the job
#         !wget -O $original_IGM_file_path $original_IGM_file_url

# # Perform validation
# Validation__tau_and_prims_limits__C  = !diff $original_IGM_file_path $outdir/apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C

# if Validation__tau_and_prims_limits__C == []:
#     # If the validation passes, we do not need to store the original IGM source code file
#     !rm $original_IGM_file_path
#     print("Validation test for apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C: PASSED!")
# else:
#     # If the validation fails, we keep the original IGM source code file
#     print("Validation test for apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C: FAILED!")
#     # We also print out the difference between the code generated
#     # in this tutorial module and the original IGM source code
#     print("Diff:")
#     for diff_line in Validation__tau_and_prims_limits__C:
#         print(diff_line)

<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [17]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
!rm -f Tut*.out Tut*.aux Tut*.log